<a href="https://colab.research.google.com/github/jeraldkim/DataScienceProjects/blob/main/Scraping_Anime_from_myanimelist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import regex as re
import time

In [ ]:
anime_list = []
num_anime = 200 #ex: 5 means 5*50 we want top 250 anime 0 would mean only top 50 anime so not run loop
n = 0

while n < num_anime:
    if n == 0:
      curr_link = "https://myanimelist.net/topanime.php"
    elif n > 0:
      curr_link = "https://myanimelist.net/topanime.php?limit=" + str(n * 50)
    n += 1

    response = requests.get(curr_link)
    html_content = response.content
    soup = BeautifulSoup(html_content, "html.parser")
    table = soup.find("table", class_="top-ranking-table")
    time.sleep(2) #necessary condition to prevent site from blocking your requests


    if table == None:
      print(curr_link)
      print(soup)
      bug1 = response
      bug2 = html_content
      bug3 = soup
    
    for row in table.find_all("tr")[1:51]:
      anime = {}
      anime["Rank"] = int(row.find("td", class_="rank").text)
      anime["Title"] = row.find("div", class_="di-ib clearfix").find("a").text
      anime["Link"] = row.find("a").get("href")
      anime["Score"] = float(row.find("td", class_="score").text)
      anime_list.append(anime)

anime_df = pd.DataFrame(anime_list)

In [ ]:
anime_df = pd.DataFrame(anime_list)
anime_df

,Rank,Title,Link,Score
0,1,Fullmetal Alchemist: Brotherhood,https://myanimelist.net/anime/5114/Fullmetal_A...,9.11
1,2,Bleach: Sennen Kessen-hen,https://myanimelist.net/anime/41467/Bleach__Se...,9.10
2,3,Steins;Gate,https://myanimelist.net/anime/9253/Steins_Gate,9.08
3,4,GintamaÂ°,https://myanimelist.net/anime/28977/GintamaÂ°,9.07
4,5,Kaguya-sama wa Kokurasetai: Ultra Romantic,https://myanimelist.net/anime/43608/Kaguya-sam...,9.07
...,...,...,...,...
9995,9996,Ultra Nyan 2: The Great Happy Operation,https://myanimelist.net/anime/4682/Ultra_Nyan_...,5.89
9996,9997,Ushinawareta Choushoku,https://myanimelist.net/anime/40171/Ushinaware...,5.89
9997,9998,World Fool News (TV),https://myanimelist.net/anime/18205/World_Fool...,5.89
9998,9999,Xiongmao Monogatari TaoTao,https://myanimelist.net/anime/24603/Xiongmao_M...,5.89


In [ ]:
from google.colab import drive

In [ ]:
# drive.mount('/content/drive')
# path = '/content/drive/My Drive/output.csv'

# with open(path, 'w', encoding = 'utf-8-sig') as f:
#   anime_df.to_csv(f)

#method to save df to csv file on google drive

0    https://myanimelist.net/anime/5114/Fullmetal_A...
1    https://myanimelist.net/anime/41467/Bleach__Se...
2       https://myanimelist.net/anime/9253/Steins_Gate
3        https://myanimelist.net/anime/28977/GintamaÂ°
4    https://myanimelist.net/anime/43608/Kaguya-sam...
5    https://myanimelist.net/anime/38524/Shingeki_n...
6    https://myanimelist.net/anime/39486/Gintama__T...
7           https://myanimelist.net/anime/9969/Gintama
8    https://myanimelist.net/anime/11061/Hunter_x_H...
9    https://myanimelist.net/anime/15417/Gintama__E...
Name: Link, dtype: object

In [96]:
new_anime_list = []

real = anime_df["Link"]

tests = anime_df["Link"][:10]

for link in tests:

    response = requests.get(link)
    html_content = response.content
    soup = BeautifulSoup(html_content, "html.parser")
    #time.sleep(2)

    anime = {}
    anime_type = soup.find("span",class_ ="dark_text",string="Type:")
    if anime_type == None:
      anime["Type"] = None
    else:
      anime["Type"] = anime_type.find_parent().find("a").text #Type of Show
    num_episodes = soup.find("span",class_ ="dark_text",string="Episodes:")
    if num_episodes == None:
      anime["# Episodes"] = None
    else:
      anime["# Episodes"] = num_episodes.next_sibling.strip() #Number of Episodes
    
    source = soup.find("span",class_ ="dark_text",string="Source:")
    if source == None:
      anime["Source"] = None
    else:
      anime["Source"] = source.next_sibling.strip() #Source

    premiered = soup.find("span",class_ ="dark_text",string="Premiered:")
    if premiered == None:
      anime["Premiered"] = None
    else:
      anime["Premiered"] = premiered.find_parent().find("a").text #Season of Anime and Year

    
    aired_date = soup.find("span",class_ ="dark_text",string="Aired:")
    if aired_date == None:
      anime["Aired Date"] = None
    else:
      anime["Aired Date"] = aired_date.next_sibling.strip() #Aired Date
    
    studio = soup.find("span",class_ ="dark_text",string="Studios:")
    if studio == None:
      anime["Studio"] = None
    else: 
      anime["Studio"] = studio.find_parent().find("a").text #Studio

    genre = soup.find("span",class_ ="dark_text",string=re.compile(r'Genres:|Genre:'))
    if genre == None:
      anime["Genre"] = None
    else:
      anime["Genre"] = genre.find_parent().find("a").text #Genre


    demo = soup.find("span",class_ ="dark_text",string="Demographic:")
    if demo == None:
      anime["Demographic"] = None
    else:
      anime["Demographic"] = demo.find_parent().find("a").text #Demographic
    
    duration = soup.find("span",class_ ="dark_text",string="Duration:")
    if duration == None:
      anime["Duration"] = None
    else:
      anime["Duration"] = duration.next_sibling.strip() #Duration

    age_rating = soup.find("span",class_ ="dark_text",string="Rating:")
    if age_rating == None:
      anime["Age Rating"] = None
    else:
      anime["Age Rating"] = age_rating.next_sibling.strip() #Age Rating

    #New feature additions started from this line
    num_raters = soup.find("span",class_ ="dark_text",string="Score:")
    if num_raters == None:
      anime["num_raters"] = None
    else:
      anime["num_raters"] = num_raters.find_parent().find(itemprop="ratingCount").text #Number of people who rated the scores

    popularity = soup.find("span",class_ ="dark_text",string="Popularity:")
    if popularity == None:
      anime["Popularity"] = None
    else:
      anime["Popularity"] = popularity.next_sibling.strip() #Popularity Rank determined by number of "members"

    members = soup.find("span",class_ ="dark_text",string="Members:")
    if members == None:
      anime["Members"] = None
    else:
      anime["Members"] = members.next_sibling.strip() # Number of members = num of  people who added the series to the list


    favorites = soup.find("span",class_ ="dark_text",string="Favorites:")
    if favorites == None:
      anime["Favorites"] = None
    else:
      anime["Favorites"] = favorites.next_sibling.strip() #Favorites 

    
    new_anime_list.append(anime)    

new_anime_df = pd.DataFrame(new_anime_list)

In [97]:
test_df = pd.DataFrame(new_anime_list)
test_df

,Type,# Episodes,Source,Premiered,Aired Date,Studio,Genre,Demographic,Duration,Age Rating,num_raters,Popularity,Members,Favorites
0,TV,64,Manga,Spring 2009,"Apr 5, 2009 to Jul 4, 2010",Bones,Action,Shounen,24 min. per ep.,R - 17+ (violence & profanity),1964276,#3,"3,089,520","213,340"
1,TV,13,Manga,Fall 2022,"Oct 11, 2022 to Dec 27, 2022",Pierrot,Action,Shounen,24 min. per ep.,R - 17+ (violence & profanity),170898,#526,"388,732","15,744"
2,TV,24,Visual novel,Spring 2011,"Apr 6, 2011 to Sep 14, 2011",White Fox,Drama,None,24 min. per ep.,PG-13 - Teens 13 or older,1304851,#13,"2,379,792","179,547"
3,TV,51,Manga,Spring 2015,"Apr 8, 2015 to Mar 30, 2016",Bandai Namco Pictures,Action,Shounen,24 min. per ep.,PG-13 - Teens 13 or older,231421,#337,"578,001","15,537"
4,TV,13,Manga,Spring 2022,"Apr 9, 2022 to Jun 25, 2022",A-1 Pictures,Comedy,Seinen,23 min. per ep.,PG-13 - Teens 13 or older,405286,#226,"756,558","27,185"
5,TV,10,Manga,Spring 2019,"Apr 29, 2019 to Jul 1, 2019",Wit Studio,Action,Shounen,23 min. per ep.,R - 17+ (violence & profanity),1417688,#27,"2,023,975","54,004"
6,Movie,1,Manga,None,"Jan 8, 2021",Bandai Namco Pictures,Action,Shounen,1 hr. 44 min.,PG-13 - Teens 13 or older,59418,#1592,"124,280","3,955"
7,TV,51,Manga,Spring 2011,"Apr 4, 2011 to Mar 26, 2012",Sunrise,Action,Shounen,24 min. per ep.,PG-13 - Teens 13 or older,220595,#384,"509,710","7,546"
8,TV,148,Manga,Fall 2011,"Oct 2, 2011 to Sep 24, 2014",Madhouse,Action,Shounen,23 min. per ep.,PG-13 - Teens 13 or older,1601231,#10,"2,576,749","195,408"
9,TV,13,Manga,Fall 2012,"Oct 4, 2012 to Mar 28, 2013",Sunrise,Action,Shounen,24 min. per ep.,PG-13 - Teens 13 or older,154063,#712,"301,161","2,927"
